In [1]:
# from nytimesarticle import articleAPI
# apikey = articleAPI('21362cb6b8374b08870bee9d1f73df92')
import requests
from datetime import datetime
from time import sleep

In [18]:
# articles = apikey.search( q = 'Russia',begin_date = 20180327 )
topic = "crime"
apikey = "21362cb6b8374b08870bee9d1f73df92"
# begin_date = "20180327"
# end_date = "20180328"
# fl = "snippet,web_url,headline"
fl = "snippet"
pageNo = "0"
dateRange = ["20180321", "20180322", "20180323", "20180324", "20180325", "20180326", "20180327", "20180328"]
# requestArticles = requests.get("http://api.nytimes.com/svc/search/v2/articlesearch.json?q="+topic+"&begin_date=20180327&end_date="+end_date+"&fl="+fl+"&page="+pageNo+"&api-key="+apikey)
# data = requestArticles.json()
# print(len(data["response"]["docs"]))
# data["response"]["docs"]
# articles['response']

In [3]:
def parse_articles(articles):
    '''
    This function takes in a response to the NYT api and parses
    the articles into a list of dictionaries
    '''
    news = []
    fetch = articles['response']['docs']
    for i in range(0,len(fetch)):
        dic = {}
#         dic['id'] = i
        if fetch[i]['snippet'] is not None:
            dic['snippet'] = fetch[i]['snippet']
#         dic['url'] = i['web_url']
        news.append(dic)
    return(news)

In [6]:
def get_articles(topic, begin_date, end_date, fl, apikey):
    all_articles = []#stores all article for a particular day
    page = 0
    while(page<100):
        
#     for page in range(0,100): #NYT limits pages to first 100 pages starting page 0, each page has 10 articles max
        try:
            
            url = "http://api.nytimes.com/svc/search/v2/articlesearch.json?q="+topic+"&begin_date="+begin_date+"&end_date="+end_date+"&fl="+fl+"&page="+str(page)+"&api-key="+apikey
            print(url)
            requestArticles = requests.get(url)
            data = requestArticles.json()
            if len(data["response"]["docs"])>0:
                all_articles.append(parse_articles(data))
            else:
                print(parse_articles(data))
                break
        except:
            print("Shit happend")
            print(data)
#             page = page - 1
            sleep(1)
            continue#try again
        print("Page: "+str(page))
#         break
        sleep(1)
        page=page+1
    return(all_articles)

In [19]:
processArticles = []
import json
import csv
for i in range(0,7):
    datetimeobject = datetime.strptime(dateRange[i],'%Y%m%d')
    beginDate = datetimeobject.strftime('%m-%d-%Y')
    datetimeobject = datetime.strptime(dateRange[i+1],'%Y%m%d')
    endDate = datetimeobject.strftime('%m-%d-%Y')
    print("Fetching articles for Data period: " + beginDate + " - "+ endDate)
    processArticles = get_articles(topic, dateRange[i], dateRange[i+1],fl, apikey)
    if(len(processArticles)>0):
        try:
            dataToWrite = processArticles[0][0:-1]
        except:
            print(len(processArticles))
            print(processArticles)
            print(processArticles[0])
            break
        with open("textcorpus/"+topic+dateRange[i]+".txt", 'w') as outfile:
            for item in dataToWrite:
                outfile.write(item["snippet"])
                outfile.write("\n")
    else:
        print("Insufficient data for date: "+beginDate+" to save")
#     break
# print(processArticles[0][0:-1])

Fetching articles for Data period: 03-21-2018 - 03-22-2018
http://api.nytimes.com/svc/search/v2/articlesearch.json?q=crime&begin_date=20180321&end_date=20180322&fl=snippet&page=0&api-key=21362cb6b8374b08870bee9d1f73df92
Page: 0
http://api.nytimes.com/svc/search/v2/articlesearch.json?q=crime&begin_date=20180321&end_date=20180322&fl=snippet&page=1&api-key=21362cb6b8374b08870bee9d1f73df92
Page: 1
http://api.nytimes.com/svc/search/v2/articlesearch.json?q=crime&begin_date=20180321&end_date=20180322&fl=snippet&page=2&api-key=21362cb6b8374b08870bee9d1f73df92
Page: 2
http://api.nytimes.com/svc/search/v2/articlesearch.json?q=crime&begin_date=20180321&end_date=20180322&fl=snippet&page=3&api-key=21362cb6b8374b08870bee9d1f73df92
Page: 3
http://api.nytimes.com/svc/search/v2/articlesearch.json?q=crime&begin_date=20180321&end_date=20180322&fl=snippet&page=4&api-key=21362cb6b8374b08870bee9d1f73df92
Page: 4
http://api.nytimes.com/svc/search/v2/articlesearch.json?q=crime&begin_date=20180321&end_date=201

In [17]:
# /usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.8.3.jar -input /user/jayant/input/ -output /user/jayant/output_new2 -mapper /home/jayant/wordcount_mapper.py -reducer /home/jayant/wordcount_reducer.py -numReduceTasks 1